In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from cplxmodule import cplx
from cplxmodule.nn import init, CplxLinear, CplxConv2d

<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

In [ ]:
def randn(shape):
    return torch.randn(shape, dtype=torch.double)

In [ ]:
sx = 32, 12, 31, 47
sw = 7, 12, 7, 11

x = cplx.Cplx(randn(sx), randn(sx))
w = cplx.Cplx(randn(sw), randn(sw))
b = cplx.Cplx(randn(sw[0]), randn(sw[0]))

# do the 2d convo manually
re = F.conv2d(x.real, w.real, bias=b.real) - F.conv2d(x.imag, w.imag, bias=None)
im = F.conv2d(x.real, w.imag, bias=b.imag) + F.conv2d(x.imag, w.real, bias=None)

# use the function from cplx
cc = cplx.conv2d(x, w, bias=b)

In [ ]:
def cplx_trabelsi_independent_(mod):
    if not hasattr(mod, "weight"):
        return

    # Trabelsi orthogonal weight initializer
    if isinstance(mod.weight, cplx):
        init.cplx_trabelsi_independent_(mod.weight)

In [ ]:
import torch
import numpy as np

from cplxmodule import Cplx
from cplxmodule.nn import init, CplxLinear, CplxConv2d


def cplx_trabelsi_independent_(mod):
    if not hasattr(mod, "weight"):
        return

    # Trabelsi orthogonal weight initializer
    if isinstance(mod.weight, Cplx):
        init.cplx_trabelsi_independent_(mod.weight)


# a model with some structure
module = torch.nn.ModuleDict(
    {
        "linear": CplxLinear(11, 17),
        "conv": CplxConv2d(13, 19, 5),
    }
).double()

# standard torch functionality `module.apply`
module.apply(cplx_trabelsi_independent_)

# according to Trabelsi et al. (2018) the reshaped weight bust be an almost unitary matrix
w = module["conv"].weight
m = w.reshape(w.shape[:2].numel(), w.shape[2:].numel()).detach().numpy()
mHm = m.conjugate().T @ m